In [ ]:
### SVM and SVR

In [1]:
# IMPORTS
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
# from sklearn.cross_validation import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR, LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
%matplotlib inline

/home/shahrokh/anaconda3/envs/research/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [11]:
# LOAD DATA
dataframe = pd.read_csv('ML/data/dynamic_pus_sensors_50000_10PUs100_100sensors_200grid_splat_202001_3100_42.txt', delimiter=',', header=None)
dataframe_max = pd.read_csv('ML/data/dynamic_pus_max_power50000_10PUs_200grid_splat_202001_3100_42.txt', delimiter=',', header=None)

dataframe.reset_index(drop=True, inplace=True)
dataframe_max.reset_index(drop=True, inplace=True)

dataframe_tot = pd.concat([dataframe, dataframe_max.iloc[:, dataframe_max.columns.values[-1]]], axis=1,
                        ignore_index=True)
idx = dataframe_tot[dataframe_tot[dataframe_tot.columns[-1]] == -float('inf')].index
dataframe_tot.drop(idx, inplace=True)

data_reg = np.concatenate((dataframe_tot.values[:, 0:dataframe_tot.shape[1]-3], 
                           dataframe_tot.values[:, dataframe_tot.shape[1]-1:dataframe_tot.shape[1]]), axis=1)
data_class = dataframe_tot.values[:, 0:dataframe_tot.shape[1]-1]
y_class_power = dataframe_tot.values[:, -1]
del dataframe, dataframe_max

In [12]:
# INIT
number_samples = [5] + list(range(10, 101, 10)) + [120, 150, 200, 250, 300, 400, 500, 700] + list(range(1000, 4001, 1000))
number_samples = [120, 150, 200, 250, 300, 400, 500, 700] + list(range(1000, 4001, 1000))
# number_samples = [6000, 8000, 10000, 12000]
C_vec = list(np.arange(0.1, 10, 0.5))
noise_floor = -90.0

In [13]:
# Splitting data
def split_data(data, train_samples):
    num_inputs = data.shape[1] - 1
    val_samples = round(train_samples/3)
    X_train, y_train = data[0:train_samples, 0: num_inputs], data[0:train_samples, -1]
    X_val, y_val = data[train_samples:train_samples+val_samples, 0: num_inputs],data[train_samples:train_samples+val_samples, -1]
    X_test, y_test = data[train_samples:, 0: num_inputs], data[train_samples:, -1]
    return X_train,X_val, X_test, y_train, y_val, y_test

def false_analysis(y_test, y_pred):
    tp = sum(y_pred[y_test==1])
    fp = sum(y_pred) - tp
    return fp, sum(y_test) - tp

In [14]:
data_reg.shape

(36249, 103)

In [ ]:
## SVM(SVC)
average_class_diff_power = []
fp_mean_power = []
accuracy, f_score, false_positive, false_negative = [], [], [], []
best_c_lst = []
fp_penalty_coef, fn_penalty_coef = 1, 1
metric = "accuracy"  # {"accuracy", "fp_min"}
class_weight = {0:fp_penalty_coef/(fp_penalty_coef + fn_penalty_coef), 1:fn_penalty_coef/(fp_penalty_coef + fn_penalty_coef)}
best_c, bestsvcclassifier = None, None
for number_sample in number_samples:
    best_accuracy = -float('inf')
    best_fp = float('inf')
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(data_class, number_sample)
    scaler_x = StandardScaler()
    scaler_x.fit(X_train)
    X_train = scaler_x.transform(X_train)
    X_val = scaler_x.transform(X_val)
    for c in C_vec:
        svclassifier = SVC(kernel='rbf', C=c, class_weight = class_weight)
        svclassifier.fit(X_train, y_train)
        
        #validating
        y_pred_val = svclassifier.predict(X_val)
        if metric == "accuracy":
            accuracy_tmp = metrics.accuracy_score(y_val, y_pred_val)
            if accuracy_tmp > best_accuracy:
                best_accuracy = accuracy_tmp
                best_c = c
                bestsvcclassifier = svclassifier
        elif metric == "fp_min":
            conf_mat = metrics.confusion_matrix(y_val, y_pred_val)
            fp_tmp = conf_mat[0][1] if len(conf_mat) == 2 else 0
            if fp_tmp < best_fp:
                best_fp = fp_tmp
                best_c = c
                bestsvcclassifier = svclassifier
                    
            
    best_c_lst.append(best_c)
    #predicting
    X_test = scaler_x.transform(X_test)
    y_pred = bestsvcclassifier.predict(X_test)
    
    #evaluating
    accuracy.append(round(metrics.accuracy_score(y_test, y_pred), 3))
    f_score.append(round(metrics.f1_score(y_true=y_test, y_pred=y_pred), 3))
    fp, fn = false_analysis(y_test, y_pred)
    false_positive.append(int(fp))
    false_negative.append(int(fn))
    
    #Power max calculations
    y_class_power_test = y_class_power[len(y_class_power)-X_test.shape[0]:]
    y_class_power_pred = np.zeros(y_class_power_test.size)
    max_power = max(y_class_power_test) + 10  # 10 is added to increase higher bound
    min_power = min(y_class_power_test) - 10  # 10 is subtracted to decrease lower bound
    for i in range(len(y_class_power_test)):
        h = max_power
        l = min_power
        while h - l > 0.5:
            mid = l + (h - l)/2;
            mid_norm = (mid - scaler_x.mean_[-1])/scaler_x.scale_[-1]
            X_test[i][-1] = mid_norm
            res_tmp = bestsvcclassifier.predict(X_test[i:i+1])
            if res_tmp[0]:
                l = mid
            else:
                h = mid
        y_class_power_pred[i] = l + (h - l)/2
    average_class_diff_power.append(round(np.mean(np.absolute(y_class_power_pred - y_class_power_test)), 3))
    fp_samples = np.zeros(len(y_class_power_pred), dtype=float)
    fp_samples[y_class_power_pred > y_class_power_test] = (y_class_power_pred - y_class_power_test)[y_class_power_pred > 
                                                                                                    y_class_power_test]
    fp_mean_power.append(round(np.mean(fp_samples), 3))
    print('Number_samples:', number_sample, ', accuracy:', accuracy[-1], ', f_score:', f_score[-1], 
          ', fp:', fp,', fn:', fn, ', error:', average_class_diff_power[-1], 'fp_error:', fp_mean_power[-1])
del svclassifier

In [ ]:
print(average_class_diff_power)
print(fp_mean_power)

In [ ]:
### MAX_POWER ANAlysis
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, accuracy)
plt.xlabel('# training samples')
plt.ylabel('%')
plt.title('SVM: Classification Accuracy(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmAcc.png')

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, f_score)
plt.xlabel('# training samples')
plt.ylabel('%')
plt.title('SVM: Classification F_score(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmfscore.png')

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, false_positive)
plt.xlabel('# training samples')
plt.ylabel('#')
plt.title('SVM: Classification FP(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmfp.png')

fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, false_negative)
plt.xlabel('# training samples')
plt.ylabel('#')
plt.title('SVM: Classification FN(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_svmfn.png')

In [ ]:
data_reg[0]

In [ ]:
data_reg[:,-1].min()

In [15]:
# SVR
average_reg_diff_power, best_c_reg_lst, fp_mean_power = [], [], []
best_c_reg, bestsvrclassifier =  None, None
# data_reg[data_reg < noise_floor] = noise_floor
# TODO: Having different penalties for fp and fn
for number_sample in number_samples:
    min_err = float('inf')
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(data_reg, number_sample)
    X_train[X_train < noise_floor] = noise_floor
    X_val[X_val < noise_floor] = noise_floor
    X_test[X_test < noise_floor] = noise_floor
    scaler_x = StandardScaler()
    scaler_x.fit(X_train)
    X_train = scaler_x.transform(X_train)
    X_val = scaler_x.transform(X_val)
    for c in C_vec:
        svrlassifier = SVR(kernel='rbf', C=c, degree=X_train.shape[1]+1)
        svrlassifier.fit(X_train, y_train)
        
        #validating
        y_pred_val = svrlassifier.predict(X_val)
        err_tmp = np.mean(np.absolute(y_pred_val - y_val))
        if err_tmp < min_err:
            min_err = err_tmp
            best_c_reg = c
            bestsvrclassifier = svrlassifier
            
    best_c_reg_lst.append(best_c_reg)
    #predicting
    X_test = scaler_x.transform(X_test)
    y_pred = bestsvrclassifier.predict(X_test)
    
    
    #evaluating
    average_reg_diff_power.append(round(np.mean(np.absolute(y_test - y_pred)), 3))
    fp_samples = np.zeros(len(y_test), dtype=float)
    fp_samples[y_pred > y_test] = (y_pred - y_test)[y_pred > y_test]
    fp_mean_power.append(round(np.mean(fp_samples), 3))
    print('Number_samples: ', number_sample, ' error: ', average_reg_diff_power[-1], ', fp_error:', fp_mean_power[-1])

Number_samples:  120  error:  6.022 , fp_error: 4.028
Number_samples:  150  error:  6.047 , fp_error: 3.936
Number_samples:  200  error:  6.025 , fp_error: 3.663
Number_samples:  250  error:  6.025 , fp_error: 3.867
Number_samples:  300  error:  6.048 , fp_error: 4.03
Number_samples:  400  error:  6.014 , fp_error: 3.834
Number_samples:  500  error:  6.009 , fp_error: 3.809
Number_samples:  700  error:  5.987 , fp_error: 3.635
Number_samples:  1000  error:  5.991 , fp_error: 3.619
Number_samples:  2000  error:  5.98 , fp_error: 3.684
Number_samples:  3000  error:  5.977 , fp_error: 3.74
Number_samples:  4000  error:  5.967 , fp_error: 3.76


In [ ]:
# Linear SVR
average_reg_diff_power, best_c_reg_lst, fp_mean_power = [], [], []
best_c_reg, bestsvrclassifier =  None, None
for number_sample in number_samples:
    min_err = float('inf')
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(data_reg, number_sample)
    scaler_x = StandardScaler()
    scaler_x.fit(X_train)
    X_train = scaler_x.transform(X_train)
    X_val = scaler_x.transform(X_val)
    for c in C_vec:
        svrlassifier = LinearSVR(C=c, loss='epsilon_insensitive')
        svrlassifier.fit(X_train, y_train)
        
        #validating
        y_pred_val = svrlassifier.predict(X_val)
        err_tmp = np.mean(np.absolute(y_pred_val - y_val))
        if err_tmp < min_err:
            min_err = err_tmp
            best_c_reg = c
            bestsvrclassifier = svrlassifier
            
    best_c_reg_lst.append(best_c_reg)
    #predicting
    X_test = scaler_x.transform(X_test)
    y_pred = bestsvrclassifier.predict(X_test)
    
    
    #evaluating
    average_reg_diff_power.append(round(np.mean(np.absolute(y_test - y_pred)), 3))
    fp_samples = np.zeros(len(y_test), dtype=float)
    fp_samples[y_pred > y_test] = (y_pred - y_test)[y_pred > y_test]
    fp_mean_power.append(round(np.mean(fp_samples), 3))
    print('Number_samples: ', number_sample, ' error: ', average_reg_diff_power[-1], ', fp_error:', fp_mean_power[-1], 
          'best_c: ', best_c_reg)

In [10]:
print(average_reg_diff_power)
print(fp_mean_power)

[5.615, 5.589, 5.579, 5.574]
[3.64, 3.635, 3.661, 3.629]


In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, average_class_diff_power)
plt.plot(number_samples, average_reg_diff_power, 'r--')
plt.xlabel('# training samples')
plt.ylabel('Diff(dB)')
plt.title('Average absolute difference power(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)

ax.set_yticks(np.arange(0,20, 2))
ax.set_ylim([2,20])
ax.set_xticks(np.arange(5,4100, 500))
# plt.grid(which='minor')
# plt.text(40, 50, '# Validation = 34k')
# plt.text(400, 45, '# Test = 34k')
plt.legend(['SVM', 'SVR'])
plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_averag_powerSVMSVR.png')

In [ ]:
average_reg_diff_power_tot = []


In [ ]:
print(number_samples)

In [ ]:
average_reg_diff_power_tot.append(average_reg_diff_power)

In [ ]:
number_samples = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150, 200, 250, 300, 400, 500, 700, 1000, 2000, 3000, 4000]
average_reg_diff_power_tot = [[4.91, 4.915, 5.02, 7.274, 5.21, 5.476, 5.547, 5.448, 5.358, 5.486, 5.424, 5.585, 5.423, 5.369,
                               5.275, 5.22, 5.065, 5.058, 4.915, 4.873, 4.8, 4.787, 4.782], 
                              [4.739, 4.728, 4.858, 4.9, 4.761, 4.835, 4.898, 4.959, 4.806, 4.913, 4.945, 4.88, 4.8, 4.812,
                               4.756, 4.74, 4.764, 4.832, 4.729, 4.745, 4.728, 4.728, 4.731], 
                              [5.59, 5.33, 4.764, 4.731, 4.728, 4.923, 5.086, 4.798, 4.816, 5.035, 4.827, 4.842, 4.757, 4.796,
                               4.734, 4.748, 4.74, 4.734, 4.728, 4.729, 4.726, 4.726, 4.728], 
                              [4.791, 4.781, 4.732, 5.677, 4.757, 4.837, 4.841, 5.242, 4.81, 4.955, 4.972, 5.106, 4.801, 4.754,
                               4.764, 4.748, 4.757, 4.763, 4.737, 4.735, 4.737, 4.74, 4.744]]

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,1,1)
plt.plot(number_samples, average_reg_diff_power_tot[0])
plt.plot(number_samples, average_reg_diff_power_tot[1], 'r--')
plt.plot(number_samples, average_reg_diff_power_tot[2], 'g.-')
plt.plot(number_samples, average_reg_diff_power_tot[3], 'y->')
plt.xlabel('# training samples')
plt.ylabel('Diff(dB)')
plt.title('Average absolute difference power(Dynamic PUs, Using PUs, Test_size=40k)')
plt.grid(True)

ax.set_yticks(np.arange(0,8, 2))
ax.set_ylim([2,8])
ax.set_xticks(np.arange(5,4100, 500))
# plt.grid(which='minor')
# plt.text(40, 50, '# Validation = 34k')
# plt.text(400, 45, '# Test = 34k')
plt.legend(['linear', 'rbf', 'poly', 'sigmoid'])
# plt.savefig('ML\\results\\changing_training_test40k_4kx4k_smallVal_compare_dynamicPUS_averag_powerSVMSVR.png')